# Loading Files

In [ ]:
import pandas as pd

X = pd.read_csv("/content/entradas_breast.csv")
y = pd.read_csv("/content/saidas_breast.csv")

In [ ]:
X.shape, y.shape

((569, 30), (569, 1))

In [ ]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1095.0000,0.9053,8589.0,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3398.0,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4585.0,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1156.0000,3445.0,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,0.7572,0.7813,5438.0,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [ ]:
y.head()

,0
0,0
1,0
2,0
3,0
4,0


# Creating the train and test data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


In [ ]:
# Checking if its ok.
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((426, 30), (426, 1), (143, 30), (143, 1))

# Creating the Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as k

In [ ]:
# Creating the network.
classifier = tf.keras.Sequential()

# Creating the hidden layer and the entry layer.
classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))

# Creating the output layer.
classifier.add(Dense(units=1, activation='sigmoid'))  # I'm gonna use the sigmoid function on the output layer to get the probability for the classes.

# Assigning the parameters.
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])   # loss function will be "binary_crossentropy", since we are facing a binary classification problem.

In [ ]:
# Traning the model

classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
43/43 [==============================] - 1s 1ms/step - loss: 7.8437 - binary_accuracy: 0.5986
Epoch 2/100
43/43 [==============================] - 0s 1ms/step - loss: 1.8173 - binary_accuracy: 0.7066
Epoch 3/100
43/43 [==============================] - 0s 2ms/step - loss: 1.0570 - binary_accuracy: 0.7817
Epoch 4/100
43/43 [==============================] - 0s 1ms/step - loss: 0.5295 - binary_accuracy: 0.8521
Epoch 5/100
43/43 [==============================] - 0s 1ms/step - loss: 0.6129 - binary_accuracy: 0.8146
Epoch 6/100
43/43 [==============================] - 0s 1ms/step - loss: 0.5798 - binary_accuracy: 0.8404
Epoch 7/100
43/43 [==============================] - 0s 1ms/step - loss: 0.3784 - binary_accuracy: 0.8897
Epoch 8/100
43/43 [==============================] - 0s 2ms/step - loss: 0.3678 - binary_accuracy: 0.8709
Epoch 9/100
43/43 [==============================] - 0s 1ms/step - loss: 0.6670 - binary_accuracy: 0.8286
Epoch 10/100
43/43 [==========================

In [ ]:
# Getting the predctions

predictions = classifier.predict(X_test)
predictions = (predictions > 0.5) # To use the sklearn metric functions I converted the probabilities results to boolean results.

In [ ]:
# Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score

score = accuracy_score(y_test, predictions)
score

0.7832167832167832

In [ ]:
matrix = confusion_matrix(y_test, predictions)
matrix

array([[24, 31],
       [ 0, 88]])

In [ ]:
# Evaluation through keras

predictions = classifier.evaluate(X_test, y_test)

5/5 [==============================] - 0s 2ms/step - loss: 1.2328 - binary_accuracy: 0.7832


# Using cross validation and GridSearch

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Dropout  # Function to prevent overfitting

####GridSearch

In [ ]:
# The creation process of the network in a function.
def create_network(optimizer, loos, kernel_initializer, activation, neurons):
    classifier = tf.keras.Sequential()
    classifier.add(Dense(units = neurons, activation = activation, 
                        kernel_initializer = kernel_initializer, input_dim = 30))
    classifier.add(Dropout(0.2))                                                       # Using the Dropout parameter to prevent overfitting.
    classifier.add(Dense(units = neurons, activation = activation, 
                        kernel_initializer = kernel_initializer))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = loos,
                      metrics = ['binary_accuracy'])
    return classifier

In [ ]:
# The model
classifier = KerasClassifier(build_fn = create_network)

# The parameters that we want to test.
# NOTE: You have to know the right actvaition functions to you specific problem. 
# Be careful not to use a function that doesn't make sense for your problem. Our problem in hand now is a binary classification problem.

parameters = {'batch_size': [10, 30],
              'epochs': [50, 100],
              'optimizer': ['adam', 'sgd'],
              'loos': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16, 8]}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)
grid_search = grid_search.fit(X, y)

Streaming output truncated to the last 5000 lines.
16/16 [==============================] - 1s 2ms/step - loss: 0.6987 - binary_accuracy: 0.4879
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6792 - binary_accuracy: 0.5758
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6760 - binary_accuracy: 0.6110
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6676 - binary_accuracy: 0.6198
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6638 - binary_accuracy: 0.6374
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6625 - binary_accuracy: 0.6396
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6554 - binary_accuracy: 0.6462
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6503 - binary_accuracy: 0.6418
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6484 - binary_accuracy: 0.6396
Epoch 1

In [ ]:
print(f'The best result is {grid_search.best_score_}')
print(f'The best parameters are{grid_search.best_params_}')

The best result is 0.9104021114733737
The best parameters are{'activation': 'relu', 'batch_size': 10, 'epochs': 100, 'kernel_initializer': 'random_uniform', 'loos': 'binary_crossentropy', 'neurons': 8, 'optimizer': 'adam'}


####Cross Validation

In [ ]:
# Creating the model with the best parameters that we get from GridSearch.

def create_network():
    k.clear_session()
    classifier = Sequential([
               tf.keras.layers.Dense(units=90, activation = 'tanh', kernel_initializer = 'uniform', input_dim=30),
               tf.keras.layers.Dense(units=90, activation = 'tanh', kernel_initializer = 'uniform'),
               tf.keras.layers.Dropout(0.3),
               tf.keras.layers.Dense(units=1, activation = 'tanh')])
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, decay = 0.0001, clipvalue = 0.5)
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
    return classifier


classifier = KerasClassifier(build_fn=create_network,
                             epochs = 500,
                             batch_size = 56)
# Cross validation
results = cross_val_score(estimator=classifier, X=X, y=y, cv=10, scoring='accuracy')

results.mean(), results.std()

# Best result that I found is:
# 2 layers with 90 units
# Actvation function tanh was best
# kernel "random_uniform" was better
# Dropout 0.3
# Optmizer Adam default
# Epochs could be more but I let 500
# batch 56 was the best that I found, you could test other value.

Streaming output truncated to the last 5000 lines.
10/10 [==============================] - 1s 4ms/step - loss: 1.6078 - binary_accuracy: 0.5312
Epoch 2/500
10/10 [==============================] - 0s 4ms/step - loss: 1.1102 - binary_accuracy: 0.6035
Epoch 3/500
10/10 [==============================] - 0s 4ms/step - loss: 0.7665 - binary_accuracy: 0.6836
Epoch 4/500
10/10 [==============================] - 0s 3ms/step - loss: 0.7195 - binary_accuracy: 0.6953
Epoch 5/500
10/10 [==============================] - 0s 4ms/step - loss: 0.5815 - binary_accuracy: 0.7305
Epoch 6/500
10/10 [==============================] - 0s 4ms/step - loss: 0.5832 - binary_accuracy: 0.7754
Epoch 7/500
10/10 [==============================] - 0s 4ms/step - loss: 0.4947 - binary_accuracy: 0.7852
Epoch 8/500
10/10 [==============================] - 0s 3ms/step - loss: 0.4711 - binary_accuracy: 0.7930
Epoch 9/500
10/10 [==============================] - 0s 4ms/step - loss: 0.4819 - binary_accuracy: 0.7734
Epoch 1

(0.8576441102756892, 0.04034698528842253)

### Creating the final model

In [ ]:
# The final neural network
k.clear_session()
classifier = Sequential([
               tf.keras.layers.Dense(units=90, activation = 'tanh', kernel_initializer = 'uniform', input_dim=30),
               tf.keras.layers.Dense(units=90, activation = 'tanh', kernel_initializer = 'uniform'),
               tf.keras.layers.Dropout(0.3),
               tf.keras.layers.Dense(units=1, activation = 'tanh')])
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, decay = 0.0001, clipvalue = 0.5)
classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
classifier.fit(X, y, batch_size = 56, epochs = 500)

Epoch 1/500
11/11 [==============================] - 1s 3ms/step - loss: 1.3310 - binary_accuracy: 0.5360
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 0.9519 - binary_accuracy: 0.6239
Epoch 3/500
11/11 [==============================] - 0s 3ms/step - loss: 0.7743 - binary_accuracy: 0.6485
Epoch 4/500
11/11 [==============================] - 0s 3ms/step - loss: 0.8158 - binary_accuracy: 0.6397
Epoch 5/500
11/11 [==============================] - 0s 3ms/step - loss: 0.6312 - binary_accuracy: 0.6731
Epoch 6/500
11/11 [==============================] - 0s 3ms/step - loss: 0.6285 - binary_accuracy: 0.7153
Epoch 7/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5817 - binary_accuracy: 0.7346
Epoch 8/500
11/11 [==============================] - 0s 3ms/step - loss: 0.6421 - binary_accuracy: 0.7030
Epoch 9/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5151 - binary_accuracy: 0.7452
Epoch 10/500
11/11 [==========================

# Saving the model

In [ ]:

# Saving the model structures.
classifier_json = classifier.to_json()
with open('classifier_breast.json', 'w') as f:
  f.write(classifier_json)

In [ ]:
# saving the model weights.
classifier.save_weights('classifier_breast.h5')

# Loading the model

In [ ]:
from tensorflow.keras.models import model_from_json

filee = open('classifier_breast.json', 'r')
network_structure = filee.read()
filee.close()

classifier = model_from_json(network_structure)
classifier.load_weights('classifier_breast.h5')